In [ ]:

資料結構
signal={'ticker', deque(signal=high/low/no_sig, timestamp, duration)}

1. get one row of data at a time;
2. capture state changes(high/low/no_sig) 定義三個states之間變化時做的事情

2-0. initialize
  if no value, add signal as no_sig

2-1.
  no_sig -> high, change(ticker, high, timestamp),
  no_sig -> low, change(ticker, low, timestamp),

2-2.
  low -> no_sig,
    if (now-timestamp)<threshold:
      change row(ticker, no_sig, ),
    else
      change row(duration),
      寫row到檔案,
      add row(ticker, no_sig,)
  high -> no_sig,
    if (now-timestamp)<threshold:
      change row(ticker, no_sig, ),
    else
      change row(duration),
      寫row到檔案,
      add row(ticker, no_sig,)

2-3.
  low -> high,
    if (now-timestamp)<threshold:
      change row(ticker, high, timestamp, ),
    else
      change row(duration),
      寫row到檔案,
      add row(ticker, high, timestamp,)

  high -> low,
    if (now-timestamp)<threshold:
      change row(ticker, low, timestamp, ),
    else
      change row(duration),
      寫row到檔案,
      add row(ticker, low, timestamp,)

即時顯示有信號的tickers及其持續期間
3. for state without duration, display in html: ticker, signal=high/low, duration=(now-timestamp)

display(signal[signal["duration"] is None])
寫到html? 若已在裡面的就更新, 瀏覽器自動refresh




In [64]:
import pickle

with open("msg_queue_240304-2.pkl", "rb") as file:
    msg_queue = pickle.load(file)
    
# msg_queue.seek(0)
display(msg_queue)

defaultdict(collections.deque,
            {'1513': deque([BidAsk(code='1513', datetime=datetime.datetime(2024, 3, 4, 10, 18, 29, 397395), bid_price=[Decimal('168'), Decimal('167.5'), Decimal('167'), Decimal('166.5'), Decimal('166')], bid_volume=[1380, 1038, 1199, 524, 870], diff_bid_vol=[0, 0, 0, 0, 0], ask_price=[Decimal('168.5'), Decimal('169'), Decimal('169.5'), Decimal('170'), Decimal('170.5')], ask_volume=[100, 167, 176, 445, 112], diff_ask_vol=[-2, 0, 0, 0, 0], suspend=0, simtrade=0, intraday_odd=0),
                    BidAsk(code='1513', datetime=datetime.datetime(2024, 3, 4, 10, 18, 29, 798520), bid_price=[Decimal('168'), Decimal('167.5'), Decimal('167'), Decimal('166.5'), Decimal('166')], bid_volume=[1380, 1038, 1199, 524, 869], diff_bid_vol=[0, 0, 0, 0, -1], ask_price=[Decimal('168.5'), Decimal('169'), Decimal('169.5'), Decimal('170'), Decimal('170.5')], ask_volume=[100, 167, 176, 445, 112], diff_ask_vol=[0, 0, 0, 0, 0], suspend=0, simtrade=0, intraday_odd=0),
             

In [65]:
# 存到檔案下次沒開盤可讀
# import pickle # Provides functions for object serialization and deserialization.
# with open("msg-queue.pkl", "wb") as file:
#     pickle.dump(msg_queue, file)

# 讀檔案 沒開盤時可試如何處理資料
import pickle
with open("msg_queue_240304-2.pkl", "rb") as file:
# with open("msg-queue-2330.pkl", "rb") as file:
    msg_queue = pickle.load(file)
# display(msg_queue)

"""
msg_queue = {'key1': deque([BidAsk(xxx=ooo,
                                   list=[360, 205, 325, 236, 208]),
                            BidAsk(xxx=ooo,
                                   list=[30, 25, 35, 26, 28]),
                            ...
                          ]),
            'key2': deque([BidAsk(xxx=ooo,
                                   list=[360, 205, 325, 236, 208]),
                            BidAsk(xxx=ooo,
                                   list=[30, 25, 35, 26, 28]),
                            ...
                          ])
            }
"""

# (觀察多檔股票, 直接輸出信號 & 偵測信號持續時間)


from collections import defaultdict, deque
from datetime import datetime, timedelta


# # new code
# monitor = deque(['1101', '1102', '1103']) # 自定義
# msg_queue = {
#     '1101': deque([1, 2, 3, 4, 5]),
#     '1102': deque([10, 20, 30, 40, 50, 60]),
#     '1103': deque([100, 100]),
# }
monitor = deque(['6290'])
# monitor = deque(['2330','8299','2368','3228','6290','7556','1513','2313'])
threshold_high = 94.71
threshold_low = 94.70
threshold_duration = timedelta(seconds=0) # e.g. days=2, hours=1, minutes=30, seconds=2
state_changes = defaultdict(list) # 存狀態改變, 結構={'ticker': [{'state_change_at': ...,'state': high/low/no_sig, 'duration': ...}, {}, ...]}
counter = 200
while counter > 0:
    """
    1. get and clean values from msg_queue
    """
    # Error handling, 1. 可容忍錯誤, 2. 不可容忍錯誤
    try:
        ticker = monitor.popleft() # 1101
    except Exception:
        raise Exception
    monitor.append(ticker)

    current_stock_queue: deque = msg_queue[ticker]  # [1, 2, 3, 4, 5]

    try:
        current_stock_information = current_stock_queue.popleft() # 1
    except Exception:
        print("no signal at", datetime.now())
        counter -= 1
        continue

    print(f"{counter}#: stock code: {ticker}, current value: {current_stock_information}")
   
    """
    2. capture state changes
        2-0. initialize
        if no value, add signal as no_sig

        2-1.
        no_sig -> high, change(ticker, high, timestamp),
        no_sig -> low, change(ticker, low, timestamp),

        2-2.
        low -> no_sig,
            if (now-timestamp)<threshold:
            change row(ticker, no_sig, ),
            else
            change row(duration),
            寫row到檔案,
            add row(ticker, no_sig,)
        high -> no_sig,
            if (now-timestamp)<threshold:
            change row(ticker, no_sig, ),
            else
            change row(duration),
            寫row到檔案,
            add row(ticker, no_sig,)

        2-3.
        low -> high,
            if (now-timestamp)<threshold:
            change row(ticker, high, timestamp, ),
            else
            change row(duration),
            寫row到檔案,
            add row(ticker, high, timestamp,)

        high -> low,
            if (now-timestamp)<threshold:
            change row(ticker, low, timestamp, ),
            else
            change row(duration),
            寫row到檔案,
            add row(ticker, low, timestamp,)
    """
    # 2-0 初始化
    if state_changes[ticker] == []:
        if current_stock_information['bid_price'][0]>=threshold_high:
            state_changes[ticker].append({'state': 'high', 'state_change_at': current_stock_information['datetime']})
        elif current_stock_information['bid_price'][0]<=threshold_low:
            state_changes[ticker].append({'state': 'low', 'state_change_at': current_stock_information['datetime']})
        else:
            state_changes[ticker].append({'state': 'no_sig'})
        continue
    # 2-3 如果信號high/low->low/high, 且大於設定時長, 就改值且加一行; 小於時長就只把state改成low/high
    elif state_changes[ticker][-1]['state'] == 'high' and current_stock_information['bid_price'][0]<=threshold_low and (datetime.now()-current_stock_information['datetime'])>=threshold_duration:
        state_changes[ticker][-1]['duration'] = datetime.now()-current_stock_information['datetime']
        state_changes[ticker].append({'state': 'low', 'state_change_at': current_stock_information['datetime']})
        continue #怕跟轉no_sig條件重複, 所以high/low互轉就跳開
    elif state_changes[ticker][-1]['state'] == 'low' and current_stock_information['bid_price'][0]>=threshold_high and (datetime.now()-current_stock_information['datetime'])>=threshold_duration:
        state_changes[ticker][-1]['duration'] = datetime.now()-current_stock_information['datetime']
        state_changes[ticker].append({'state': 'high', 'state_change_at': current_stock_information['datetime']})
        continue
    elif state_changes[ticker][-1]['state'] == 'high' and current_stock_information['bid_price'][0]<=threshold_low and (datetime.now()-current_stock_information['datetime'])<threshold_duration:
        state_changes[ticker][-1]['state']= 'low'
        continue
    elif state_changes[ticker][-1]['state'] == 'low' and current_stock_information['bid_price'][0]>=threshold_high and (datetime.now()-current_stock_information['datetime'])<threshold_duration:
        state_changes[ticker][-1]['state']= 'high'
        continue
    # 2-2 如果信號high/low->no_sig, 且大於設定時長, 就改值且加一行; 小於時長就只把state改回'no_sig
    elif state_changes[ticker][-1]['state'] == 'high' and current_stock_information['bid_price'][0]<threshold_high and (datetime.now()-current_stock_information['datetime'])>=threshold_duration:
        state_changes[ticker][-1]['duration'] = datetime.now()-current_stock_information['datetime']
        state_changes[ticker].append({'state': 'no_sig'})
    elif state_changes[ticker][-1]['state'] == 'low' and current_stock_information['bid_price'][0]>threshold_low and (datetime.now()-current_stock_information['datetime'])>=threshold_duration:
        state_changes[ticker][-1]['duration'] = datetime.now()-current_stock_information['datetime']
        state_changes[ticker].append({'state': 'no_sig'})
    elif state_changes[ticker][-1]['state'] == 'high' and current_stock_information['bid_price'][0]<threshold_high and (datetime.now()-current_stock_information['datetime'])<threshold_duration:
        state_changes[ticker][-1]['state']= 'no_sig'
    elif state_changes[ticker][-1]['state'] == 'low' and current_stock_information['bid_price'][0]>threshold_low  and (datetime.now()-current_stock_information['datetime'])<threshold_duration:
        state_changes[ticker][-1]['state']= 'no_sig'
    # 2-1 如果信號no_sig->high/low, 就改值
    elif state_changes[ticker][-1]['state'] == 'no_sig' and current_stock_information['bid_price'][0]>=threshold_high:
        state_changes[ticker][-1]['state']= 'high'
        state_changes[ticker][-1]['state_change_at'] = current_stock_information['datetime']
    elif state_changes[ticker][-1]['state'] == 'no_sig' and current_stock_information['bid_price'][0]<=threshold_low:
        state_changes[ticker][-1]['state']= 'low'
        state_changes[ticker][-1]['state_change_at'] = current_stock_information['datetime']

    # do something with current_stock_information
    # filter_set() .....
    # filter_status() ....
    #
    counter -= 1


print(state_changes)

200#: stock code: 6290, current value: BidAsk(code='6290', datetime=datetime.datetime(2024, 3, 4, 10, 18, 31, 399080), bid_price=[Decimal('94.6'), Decimal('94.5'), Decimal('94.4'), Decimal('94.3'), Decimal('94.2')], bid_volume=[16, 41, 12, 53, 65], diff_bid_vol=[0, 0, 0, 0, 0], ask_price=[Decimal('94.8'), Decimal('94.9'), Decimal('95'), Decimal('95.1'), Decimal('95.2')], ask_volume=[9, 3, 45, 31, 25], diff_ask_vol=[0, 0, 1, 0, 0], suspend=0, simtrade=0, intraday_odd=0)
200#: stock code: 6290, current value: BidAsk(code='6290', datetime=datetime.datetime(2024, 3, 4, 10, 18, 33, 67745), bid_price=[Decimal('94.6'), Decimal('94.5'), Decimal('94.4'), Decimal('94.3'), Decimal('94.2')], bid_volume=[16, 41, 12, 53, 65], diff_bid_vol=[0, 0, 0, 0, 0], ask_price=[Decimal('94.8'), Decimal('94.9'), Decimal('95'), Decimal('95.1'), Decimal('95.2')], ask_volume=[10, 3, 45, 31, 25], diff_ask_vol=[1, 0, 0, 0, 0], suspend=0, simtrade=0, intraday_odd=0)
199#: stock code: 6290, current value: BidAsk(code='

In [ ]:
from datetime import datetime, timedelta

print(datetime.now())
print(current_stock_information['datetime'])
delta=current_stock_information['datetime']-datetime.now()
print(delta)

print(delta>=timedelta(days=2))


2024-03-04 17:39:34.921689
2024-02-16 09:16:26.899720
-18 days, 15:36:51.978031
False


In [ ]:
counter = 7
list = deque([1,2,3])
while counter > 0:
    code = list.popleft()
    print(code)
    list.append(code)
    counter -= 1

# Gemini: 多加一行到csv檔案最後
import pandas as pd
data = ["item1", "item2", "item3"]
df = pd.read_csv("your_file.csv")
df = df.append(pd.DataFrame([data]), ignore_index=True)  # Add as new row
df.to_csv("your_file.csv", index=False)  # Save without index

1
2
3
1
2
3
1


In [ ]:
from datetime import datetime, timedelta

print(datetime.now())
print(current_stock_information['datetime'])
delta=current_stock_information['datetime']-datetime.now()
print(delta)

print(delta>=timedelta(days=2))


2024-03-04 17:39:34.921689
2024-02-16 09:16:26.899720
-18 days, 15:36:51.978031
False
